## Import libraries

In [15]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from lenet5 import LeNet
import matplotlib
import matplotlib.pyplot as plt

## Set Parameters 

In [16]:
# Check if cuda is available
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model Training Parameters
EPOCHS = 100
LEARNING_RATE = 2e-3
BATCH_SIZE = 32

# Save plots in the folder ./plots or show them
SAVE_PLOTS = False


# Use batch normalization or not
BATCH_NORM = False 
# group normalization
GROUP_NORM = True 

# group normalization parameters
groupNormParams= {
'groupNL1' : 2,
'groupNL2' :2
}


if GROUP_NORM ==True & BATCH_NORM ==True:
    print(" Cannot have group an batch normalization True at the same time")
    exit()


## Training and Validation functions

In [17]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for X, y_target in train_loader:

        # Set gradient to zero
        optimizer.zero_grad()

        # If there is a GPU, pass the data to the GPU

        X = X.to(device)
        y_target = y_target.to(device)

        # Prediction

        # Call model forward()
        y_predict, _ = model(X)

        # Get loss
        loss = criterion(y_predict, y_target)
        running_loss += loss.item() * X.size(0)

        # Adjusting weights
        loss.backward()
        optimizer.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss


def test(valid_loader, model, criterion, device):
    model.eval()
    running_loss = 0

    for X, y_target in valid_loader:
        # If there is a GPU

        X = X.to(device)
        y_target = y_target.to(device)

        # Prediction and loss

        # Call model forward()
        y_predict, _ = model(X)
        
        # Get loss
        loss = criterion(y_predict, y_target)
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
    return model, epoch_loss


def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''

    correct_pred = 0
    n = 0

    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n


In [18]:
def training_loop(model, criterion, optimizer, train_loader, test_loader,
                  epochs, device, print_every=1):

    train_losses = []
    valid_losses = []

    train_accuracies = []
    valid_accuracies = []

    # Train model
    for epoch in range(epochs):
        model, optimizer, train_loss = train(train_loader, model,
                                             criterion, optimizer, device)
        train_losses.append(train_loss)

        # Validation

        # Disable gradient calculation to save memory
        with torch.no_grad():
            model, valid_loss = test(test_loader, model, criterion, device)
            valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):
            train_acc = get_accuracy(model, train_loader, device)
            valid_acc = get_accuracy(model, test_loader, device)

            train_accuracies.append(train_acc)
            valid_accuracies.append(valid_acc)

            print(f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')

    return model, optimizer, (train_losses, valid_losses), (train_accuracies, valid_accuracies)


## Loaders and Transformations

In [19]:
# Image augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalizing the test images
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Loading CIFAR10 using torchvision.datasets
traindata = datasets.CIFAR10('./data', train=True, download=False,
                             transform=transform_train)

# Creating a pytorch loader for a Deep Learning model
train_loader = torch.utils.data.DataLoader(
    traindata, batch_size=BATCH_SIZE, shuffle=True)

# Loading the test images and thus converting them into a test_loader
test_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./data', train=False,
                                                           transform=transforms.Compose([transforms.ToTensor(),
                                                                                         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
                                                           ), batch_size=BATCH_SIZE, shuffle=True)


## Model Selection

In [20]:
model = LeNet(BATCH_NORM,GROUP_NORM,groupNormParams).to(DEVICE)

optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()


## Train

In [21]:
model, optimizer, (train_losses, valid_losses), (train_accuracies, valid_accuracies) = training_loop(model, criterion, optimizer,
                                                                                                     train_loader, test_loader, EPOCHS, DEVICE)


Epoch: 0	Train loss: 2.0217	Valid loss: 1.7309	Train accuracy: 36.16	Valid accuracy: 38.99
Epoch: 1	Train loss: 1.6558	Valid loss: 1.4766	Train accuracy: 43.51	Valid accuracy: 46.31
Epoch: 2	Train loss: 1.4880	Valid loss: 1.3472	Train accuracy: 47.37	Valid accuracy: 50.51
Epoch: 3	Train loss: 1.3784	Valid loss: 1.2484	Train accuracy: 50.94	Valid accuracy: 54.49
Epoch: 4	Train loss: 1.2999	Valid loss: 1.4648	Train accuracy: 44.78	Valid accuracy: 49.43
Epoch: 5	Train loss: 1.2380	Valid loss: 1.1406	Train accuracy: 54.87	Valid accuracy: 58.09
Epoch: 6	Train loss: 1.1873	Valid loss: 1.0942	Train accuracy: 56.83	Valid accuracy: 60.89
Epoch: 7	Train loss: 1.1495	Valid loss: 1.0431	Train accuracy: 59.36	Valid accuracy: 62.19
Epoch: 8	Train loss: 1.1121	Valid loss: 1.0168	Train accuracy: 59.82	Valid accuracy: 64.37
Epoch: 9	Train loss: 1.0839	Valid loss: 1.0434	Train accuracy: 59.29	Valid accuracy: 62.31
Epoch: 10	Train loss: 1.0585	Valid loss: 0.9456	Train accuracy: 63.09	Valid accuracy: 66.8

## Plot Accuracy and Loss

In [24]:
SAVE_PLOTS = False
if SAVE_PLOTS:
    matplotlib.use("Agg")

# Plot Loss
plt.figure()
plt.title("Training and Validation Loss")
plt.plot(range(len(train_losses)), train_losses, color="r", label="Training")
plt.plot(range(len(valid_losses)), valid_losses, color="b", label="Validation")
plt.legend(loc="best")
plt.ylabel("Loss")
plt.xlabel("Epochs")
if SAVE_PLOTS:
    plt.savefig(
        "plots/centralised_LR[{}]_EPOCHS[{}]_BATCHNORM[{}]_GROUPNORM[{},{},{}]_loss.png".format(LEARNING_RATE, EPOCHS, BATCH_NORM, GROUP_NORM, groupNormParams['groupNL1'], groupNormParams['groupNL2']))
else:
    plt.show()

# Plot Accuracy
plt.figure()
plt.title("Training and Validation Accuracy")
plt.plot(range(len(train_accuracies)),
         train_accuracies, color="r", label="Training")
plt.plot(range(len(valid_accuracies)), valid_accuracies,
         color="b", label="Validation")
plt.legend(loc="best")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
if SAVE_PLOTS:
    plt.savefig(
        "plots/centralised_LR[{}]_EPOCHS[{}]_BATCHNORM[{}]_GROUPNORM[{},{},{}]_accuracy.png".format(LEARNING_RATE, EPOCHS, BATCH_NORM, GROUP_NORM, groupNormParams['groupNL1'], groupNormParams['groupNL2']))
else:
    plt.show()


/var/folders/m1/rvjhdxrd0jjg1f2vnlnwm7y40000gn/T/ipykernel_5201/1643168129.py:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/m1/rvjhdxrd0jjg1f2vnlnwm7y40000gn/T/ipykernel_5201/1643168129.py:33: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
